# Week 1

In [2]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [33]:
import numpy as np
from functools import reduce
import string
import nltk
from nltk.corpus import stopwords
import math
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Read file and re-processing

In [6]:
def re_processing_words(vocab):
  process_words = []
  for line in vocab:
    splitted_line = line.split()[:-1] # removing sign '/' at the end of the line

    for word in splitted_line:
      if word.strip() and not word.isdigit():
        process_words.append(word.lower())
  process_words.sort()
  return process_words

def remove_stop_words(docs):
  english_stop_words = stopwords.words('english')
  new_docs = []
  for doc in docs:
    new_docs.append(' '.join([word for word in doc.split() if word not in english_stop_words]))
  return new_docs


def re_processing_docs(docs):
  processed_docs = []
  # removing digit and break line ('\n)
  st1_pr_docs = list(filter(lambda y: not y.isdigit(),map(lambda x: x[:-1], docs)))
  # splitting allow forward flash
  table_remove_punctuation = str.maketrans(dict.fromkeys(string.punctuation))
  docs_tmp = []
  for doc in st1_pr_docs:
    if doc.endswith('/'):
      processed_docs.append(' '.join(docs_tmp))
      docs_tmp.clear()
    else:
      docs_tmp.append(' '.join([w.lower().translate(table_remove_punctuation) for w in doc.split()]))
  return processed_docs

In [7]:
vocab = None 
with open('./dataset/term-vocab', 'r') as f:
  vocab = f.readlines()
processed_vocab = re_processing_words(vocab)
print(processed_vocab[-1])

zurich


In [8]:
docs = None
with open('./dataset/doc-text', 'r') as f:
  docs = f.readlines()
processed_docs = remove_stop_words(re_processing_docs(docs))
print(processed_docs[0])

compact memories flexible capacities digital data storage system capacity bits random sequential access described


## Creating Marked matrix

In [9]:
# initializing
marked_matrix = np.zeros(shape=(len(processed_vocab), len(processed_docs)), dtype=np.uint8)
# marked
for w, word in enumerate(processed_vocab):
  for d, doc in enumerate(processed_docs):
    if word in doc:
      marked_matrix[w, d] = 1

print(marked_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# lookup index table
table_vocab_idx = {}
for i, word in enumerate(processed_vocab):
  table_vocab_idx[word] = i

print(table_vocab_idx)

## Creating Reverted Index

In [ ]:
# initializing
reverted_index = reduce(lambda acc, word: {**acc, word: []} , list(set([word for word in (' '.join(processed_docs)).split() if word.strip()])), {})
reverted_index

In [50]:
for i, doc in enumerate(processed_docs):
  for word in list(set([word for word in doc.split() if doc.strip()])):
    reverted_index[word].append(i + 1)

for key in reverted_index:
    reverted_index[key] = list(set(reverted_index[key]))
    reverted_index[key].sort()

list_tmp = list(reverted_index.items())
list_tmp.sort(key=lambda x: x[0])

reverted_index = dict(list_tmp)
with open('./result/reverted_index.txt', 'w') as f:
  for key, value in reverted_index.items():
    f.write(str(key) + '\n' + ','.join(map(lambda x: str(x), value)) + '\n\n/')

# Query

### Re-processing query docs

In [51]:
# re-processing query doc
query_docs = None
with open('./dataset/query-text', 'r') as f:
  query_docs = f.readlines()

processed_query_docs = remove_stop_words(re_processing_docs(query_docs))
print(processed_query_docs[0])

measurement dielectric constant liquids use microwave techniques


### Query with marked matrix

In [53]:
tmp = []
with open('./result/marked_matrix.txt', 'w') as f:
  for i, query_doc in enumerate(processed_query_docs):
    f.write(str(i + 1) + '\n')
    tmp.clear()
    for query_word in query_doc.split():
      if query_word in processed_vocab:
        tmp.append(marked_matrix[table_vocab_idx.get(query_word)])

    if tmp:
      answers = reduce(lambda acc, x: np.bitwise_and(np.array(acc), np.array(x), dtype=np.uint8), tmp)
      for i, isMarked in enumerate(answers):
        if isMarked == 1:
          f.write(str(i + 1) + ',')
    f.write('\n/\n')
  
  f.close()


### Query with reverted index

In [54]:
def intersection(posting_list1, posting_list2):
  doc_id_list = []
  i_pl1, i_pl2 = 0, 0
  len_pl1 = len(posting_list1)
  len_pl2 = len(posting_list2)
  while(i_pl1 < len_pl1 and i_pl2 < len_pl2):
    if posting_list1[i_pl1] == posting_list2[i_pl2]:
      doc_id_list.append(posting_list1[i_pl1])
      i_pl1 += 1
      i_pl2 += 1
    elif posting_list1[i_pl1] < posting_list2[i_pl2]:
      i_pl1 += 1
    else:
      i_pl2 += 1
  return doc_id_list

def print_answer(query):
  def print_result(doc_id_answers):
    print('The docIDs satisfied the query: ', query)
    if not doc_id_answers:
      print('Don\'t have any answers')
    for doc_id in doc_id_answers:
      print(doc_id)
  return print_result

### Optimizing (sort with df)

In [ ]:

list_tmp = list(reverted_index.items())
list_tmp.sort(key=lambda x: len(x[1]))
list_tmp

In [57]:
keys = list(reverted_index.keys())
with open('./result/query_reverted_index.txt', 'w') as f:
  for i, query_doc in enumerate(processed_query_docs):
    f.write(str(i + 1) + ' ')
    query_words = [word for word in query_doc.split() if word.strip() and word in keys]

    intersect = reverted_index[query_words[0]]
    query_words = query_words[1:]

    print_result = print_answer(query_doc)

    for word in query_words:
      intersect = intersection(intersect, reverted_index[word])
    
    f.write(str(','.join(map(lambda x: str(x), intersect)) + '\n'))
    f.write('/\n')

    print_result(intersect)

The docIDs satisfied the query:  measurement dielectric constant liquids use microwave techniques
Don't have any answers
The docIDs satisfied the query:  mathematical analysis design details waveguide fed microwave radiations
Don't have any answers
The docIDs satisfied the query:  use digital computers design band pass filters given phase attenuation characteristics
Don't have any answers
The docIDs satisfied the query:  systems data coding information transfer
Don't have any answers
The docIDs satisfied the query:  use programs engineering testing computers
Don't have any answers
The docIDs satisfied the query:  number representation binary machines
Don't have any answers
The docIDs satisfied the query:  secondary emission electrons positive ion bombardment cathode
Don't have any answers
The docIDs satisfied the query:  measurement plasma temperatures arc discharge using shock wave techniques
Don't have any answers
The docIDs satisfied the query:  characteristics single electrode disc

# Skip pointer

In [26]:
def has_skip(skip_step, len_list):
  def check(p_list):
    return (skip_step + p_list) < len_list
  return check

def intersection_with_skip_pointer(list1, list2, skip_step = 1):
  answer = []
  p_list1, p_list2 = 0, 0
  len_p_list1 = len(list1)
  len_p_list2 = len(list2)

  has_skip_list1 = has_skip(skip_step, len_p_list1)
  has_skip_list2 = has_skip(skip_step, len_p_list2)

  while p_list1 < len_p_list1 and p_list2 < len_p_list2:
    if list1[p_list1] == list2[p_list2]:
      answer.append(list1[p_list1])
      p_list1 += 1
      p_list2 += 1
    elif list1[p_list1] < list2[p_list2]:
      if has_skip_list1(p_list1) and list1[p_list1 + skip_step] <= list2[p_list2]:
        while has_skip_list1(p_list1) and list1[p_list1 + skip_step] <= list2[p_list2]:
          p_list1 += skip_step
      else: p_list1 += 1
    else:
      if has_skip_list2(p_list2) and list2[p_list2 + skip_step] <= list1[p_list1]:
        while has_skip_list2(p_list2) and list2[p_list2 + skip_step] <= list1[p_list1]:
          p_list2 += skip_step
      else: p_list2 += 1
  return answer

In [29]:
# testing
intersection_with_skip_pointer([2,5,8,41,48,64,128], [1,2,3,8,11,17,21,31], 3)

[2, 8]

In [58]:
keys = list(reverted_index.keys())
with open('./result/query_reverted_index_skip_pointer.txt', 'w') as f:
  for i, query_doc in enumerate(processed_query_docs):
    f.write(str(i + 1) + ' ')
    query_words = [word for word in query_doc.split() if word.strip() and word in keys]

    intersect = reverted_index[query_words[0]]
    query_words = query_words[1:]

    print_result = print_answer(query_doc)

    for word in query_words:
      intersect = intersection_with_skip_pointer(intersect, reverted_index[word], int(math.sqrt(len(reverted_index[word]))))
    
    f.write(str(','.join(map(lambda x: str(x), intersect))) + '\n')
    f.write('/\n')

    print_result(intersect)

The docIDs satisfied the query:  measurement dielectric constant liquids use microwave techniques
Don't have any answers
The docIDs satisfied the query:  mathematical analysis design details waveguide fed microwave radiations
Don't have any answers
The docIDs satisfied the query:  use digital computers design band pass filters given phase attenuation characteristics
Don't have any answers
The docIDs satisfied the query:  systems data coding information transfer
Don't have any answers
The docIDs satisfied the query:  use programs engineering testing computers
Don't have any answers
The docIDs satisfied the query:  number representation binary machines
Don't have any answers
The docIDs satisfied the query:  secondary emission electrons positive ion bombardment cathode
Don't have any answers
The docIDs satisfied the query:  measurement plasma temperatures arc discharge using shock wave techniques
Don't have any answers
The docIDs satisfied the query:  characteristics single electrode disc